In [ ]:
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

<FONT SIZE=6><strong>ANALISIS COMPUTACIONAL: MODELO BUCHOWSKI-KSIAZCZAK
  (Ecuación - $\lambda h$)

  Despejando la variable $T$ en el módelo obtenemos:

$$T = \dfrac{1}{\dfrac{\ln \left[ 1+ \dfrac{\lambda (1 -x)}{x} \right]}{\lambda h} + \dfrac{1}{T_m}}$$


Despejando la variable $x$ en el módelo obtenemos:

$$x =\dfrac{\lambda e^{\frac{\lambda h}{T_m}}}{\lambda e^{\frac{\lambda h}{T_m}}-e^{\frac{\lambda h}{T_m}}+e^{\frac{\lambda h}{T}}}$$


In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Math, Latex, Markdown,HTML
from IPython.core.display import display, HTML
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from google.colab import files
from ipykernel import kernelapp as app
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


class dataframe:
    
    def __init__(self,url):
            self.url =url

    @property
    def show(self):
        df = pd.read_csv(self.url)
        return df
    
    @property
    def temperature_values(self):
        df = self.show
        temp = pd.DataFrame(df["T"])
        return temp
 
    @property
    def name_mf(self):
        df = self.show
        name_mf = list(df.columns[1:])
        return name_mf
    

    def mole_fractions(self, mf, temp = True):
        df = self.show

        if temp == True:
            data_mole = pd.concat([self.temperature_values,df[mf]],axis=1)
        else:
            data_mole = pd.DataFrame(df[mf])
        return data_mole

    def experimental_values(self,temp = True):

        W = self.name_mf

        C_data = []

        if temp == True:
            for i in W:
                C_data.append(self.mole_fractions(i).rename({i: 'x_exp'}, axis=1))
        else:
            for i in W:
                C_data.append(self.mole_fractions(i,temp = False).rename({i: 'x_exp'}, axis=1))

        DF = pd.concat(C_data,axis=0,ignore_index=True)
        return DF



#CLASE PARA EL MODELO DE SOLUBILIDAD λh

class Buchowski_Ksiazaczak_model(dataframe):
    
    @property
    def equation(self):
        salida = display(HTML('<h2> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Buchowski-Ksiazczak Equation</h2>'))
        display(Math(r'$$\ln \left[ 1+ \dfrac{\lambda (1 -x)}{x} \right] = \lambda h \left( \dfrac{1}{T} - \dfrac{1}{T_m}  \right)$$'))
        return salida

 
    def parameters(self,Tm, funtion = "fx", method="lm", maxfev=20000, sd = True):

        def fx(x,λ,h):
            return 1/((np.log(1+(λ*(1-x)/x))/(λ*h))+1/tm)

        def fT1(T,λ,h):
            return (λ*np.exp(λ*h/tm))/(λ*np.exp(λ*h/tm)-np.exp(λ*h/tm)+np.exp(λ*h/T))

        def fT2(T,λ,h):
            return (λ*np.exp(h*λ*(T-tm)/(tm*T)))/(((λ-1)*np.exp(h*λ*(T-tm)/(tm*T)))+1)


        df = self.show
        
        dic_Tm = {"progesterone":403.4,"sulfamerazine":508.45, "sulfamethazine":468.95, "sulfadiazine" :302.985}
        tm = dic_Tm[Tm]

        para_λ = []
        para_h = []

        desv_para_λ = []
        desv_para_h = []

        if funtion== "fx":
            
            for i in self.name_mf:
                xdat = self.mole_fractions(i)[i]
                Tdat = self.mole_fractions(i)["T"]
                popt, mcov= curve_fit(fx,xdat,Tdat,method= "lm",maxfev=20000)

                para_λ.append(popt[0])
                para_h.append(popt[1])

                desv_para_λ.append(np.sqrt((np.diag(mcov))[0]))
                desv_para_h.append(np.sqrt((np.diag(mcov))[1]))
        
        if funtion == "fT1":

            for i in self.name_mf:
                xdat = self.mole_fractions(i)[i]
                Tdat = self.mole_fractions(i)["T"]
                popt, mcov= curve_fit(fT1,Tdat,xdat,method= "lm",maxfev=20000)

                para_λ.append(popt[0])
                para_h.append(popt[1])

                desv_para_λ.append(np.sqrt((np.diag(mcov))[0]))
                desv_para_h.append(np.sqrt((np.diag(mcov))[1]))

        if funtion == "fT2":

            for i in self.name_mf:
                xdat = self.mole_fractions(i)[i]
                Tdat = self.mole_fractions(i)["T"]
                popt, mcov= curve_fit(fT2,Tdat,xdat,method= "lm",maxfev=20000)

                para_λ.append(popt[0])
                para_h.append(popt[1])

                desv_para_λ.append(np.sqrt((np.diag(mcov))[0]))
                desv_para_h.append(np.sqrt((np.diag(mcov))[1]))


        if sd == True:
            DF = pd.DataFrame({"x":self.name_mf,'λ':para_λ,"σ_λ":desv_para_λ,'h':para_h,"σ_h":desv_para_h})
        else:
            DF = pd.DataFrame({"x":self.name_mf,'λ':para_λ,'h':para_h})
        return  DF



    def __generator_λ(self,Tm,method = "lm", maxfev = 20000):
        para_λ = self.parameters(Tm,method = method, maxfev =maxfev,sd = False)["λ"]
        for i in para_λ:
            yield i

    def __generator_h(self,Tm,method = "lm", maxfev = 20000):
        para_h = self.parameters(Tm,method = method, maxfev =maxfev,sd = False)["h"]
        for i in para_h:
            yield i


    def parameters_λ(self,Tm,method = "lm", maxfev = 20000):
        para_λ = [i for i in modelo.__generator_λ(Tm,method =method, maxfev = maxfev)]
        dataframe = pd.DataFrame({"x":self.name_mf,'λ':para_λ})
        return dataframe


    def parameters_h(self,Tm,method = "lm", maxfev = 20000,temp = True):
        para_h =[i for i in modelo.__generator_h(Tm,method =method, maxfev = maxfev)]
        dataframe = pd.DataFrame({"x":self.name_mf,'h':para_h})
        return  dataframe


    def calculated_values(self,Tm, method="lm", maxfev=20000, temp = True, RD = False):

        dic_Tm = {"progesterone":403.4,"sulfamerazine":508.45, "sulfamethazine":468.95, "sulfadiazine" :302.985}
        tm = dic_Tm["progesterone"]

        def fx(x,λ,h):
            return 1/((np.log(1+(λ*(1-x)/x))/(λ*h))+1/tm)
    
        def fT1(T,λ,h):
            return (λ*np.exp(λ*h/tm))/(λ*np.exp(λ*h/tm)-np.exp(λ*h/tm)+np.exp(λ*h/T))

        def fT2(T,λ,h):
            return (λ*np.exp(h*λ*(T-tm)/(tm*T)))/(((λ-1)*np.exp(h*λ*(T-tm)/(tm*T)))+1)
       
        df = pd.read_csv(self.url)
        W  = self.name_mf

        para_λ = []
        para_h = []
      
        C_temp = []
        C_cal  = []
        C_RD   = []

        for i in W:
            tdat = df["T"].values
            xdat = df[i].values
            popt,mcov = curve_fit(fx,xdat,tdat,method=method,maxfev=maxfev)

            para_λ.append(popt[0])
            para_h.append(popt[1])
          

        for i in W:
            tdat = df["T"].values
            Tdat = tdat.tolist()

            xdat = df[i].values
            Xdat =  xdat.tolist()

            xcal = fT2(tdat,para_λ[W.index(i)],para_h[W.index(i)])
            Xcal = xcal.tolist()

            Relative_differences = ((xdat - xcal)/xdat).tolist()


            C_temp += Tdat
            C_cal  += Xcal
            C_RD   += Relative_differences
 

        arr_tem = np.array(C_temp)
        arr_cal = np.array(C_cal)
        arr_RD  = np.array(C_RD )
        
        if  temp == True  and RD == False:
            data_frame = pd.DataFrame({'T': arr_tem,'x_cal':arr_cal})
        elif temp == True and RD == True:
            data_frame = pd.DataFrame({'T': arr_tem,'x_cal':arr_cal,'RD':arr_RD})
        elif temp == False and RD == False:
            data_frame = pd.DataFrame({'x_cal':arr_cal})
        elif temp == False and RD == True:
            data_frame = pd.DataFrame({'x_cal':arr_cal,'RD':arr_RD})
        return data_frame
        
    def values(self,Tm, method="lm", maxfev=20000, temp = True,RD =True):
        
        df_exp = self.experimental_values(temp = False)["x_exp"]
        df_cal = self.calculated_values(Tm, method=method, maxfev=maxfev, temp = False, RD = RD)
        
        if temp == True:
            df_tem = self.experimental_values(temp = temp)["T"]
            data_frame = pd.concat([df_tem,df_exp,df_cal],axis=1)
        else:
            data_frame = pd.concat([df_exp,df_cal],axis=1)
        return data_frame

    def statistic_MAPE(self,Tm, method="lm", maxfev=20000, temp = True, data= "temperature", value = False):

        Temp = self.temperature_values["T"].values
        df_valuecal = self.calculated_values(Tm, method=method, maxfev=maxfev, temp = True, RD = True)

        MAPES = []
        INDEX = []

        for i in range(len(Temp)):

            df_mask = df_valuecal['T'] == Temp[i]
            data_filter = df_valuecal[df_mask]
            MAPE = sum(abs(data_filter["RD"]))*100/len(data_filter['T'])
            MAPES.append(MAPE)
            INDEX.append("T"+str(i+1))

        if data == "temperature":
            if temp == True:
                data_mape = pd.DataFrame({"T":INDEX,"Temp":Temp,"MAPE_"+Tm:MAPES})
            if  temp == False:
                data_mape = pd.DataFrame({"MAPE_"+Tm:MAPES})
        if data == "all":
            if value == False:
                data_mape = print("Mean Absolute Percentage Error, MAPE_"+Tm+" = ",sum(MAPES)/len(Temp))
            if value == True:
                data_mape = sum(MAPES)/len(Temp)
        return data_mape

    def statistic_RMSD(self,Tm,method="lm", maxfev=20000,value = False):

        df_values = self.values(Tm, method=method, maxfev=maxfev, temp = False, RD = True)
    
        ss_res = np.sum((df_values["x_cal"] - df_values["x_exp"])**2 )
        RMSD = np.sqrt(ss_res/len(df_values["x_exp"]))

        if value == False:
            statistic = print("Root Mean Square Deviation, RMSD_"+Tm+" = ",RMSD )
        if value ==True:
            statistic = RMSD

        return statistic

    def statistic_AIC(self,Tm,method="lm", maxfev=20000, corrected = True,value = False):

        df_values = self.values(Tm, method=method, maxfev=maxfev, temp = False, RD = True)

        k = 3
        N =len(df_values["RD"])
        ss_res = np.sum((df_values["x_cal"] - df_values["x_exp"])**2 )
        AIC = N*np.log(ss_res/N)+2*k
        
        AICc = abs(AIC +((2*k**2+2*k)/(N-k-1)))
        
        if corrected == True:
            if value == False:
                aic = print("Akaike Information Criterion corrected , AICc_"+Tm+" = ",AICc)
            if value == True:
                aic = AICc
        if corrected == False:
            if value == False:
                aic = print("Akaike information criterion, AIC_"+Tm+" = ",AIC )
            if value == True:
                aic = AIC
        return aic

    def statistic_RS(self,Tm,method="lm", maxfev=20000,value = False):

        df_values = self.values(Tm, method=method, maxfev=maxfev, temp = False, RD = True)
        
        ss_res = np.sum((df_values["x_cal"] - df_values["x_exp"])**2 )
        ss_tot = np.sum( (df_values["x_exp"] - np.mean(df_values["x_exp"]) )**2  )

        R2 = 1 - (ss_res / ss_tot)

        if value == False:
            statistic =print("Coefficient of Determination, R^2_"+Tm+" = ",R2)
        if value == True:
            statistic = R2
            
        return statistic

    def statistics(self,Tm,method="lm", maxfev=20000):

        
        MAPE = self.statistic_MAPE(Tm,method=method, maxfev=maxfev,data = "all",value = True)
        RMSD = self.statistic_RMSD(Tm,method=method, maxfev=maxfev,value = True)
        AIC  = self.statistic_AIC(Tm,method=method, maxfev=maxfev,value = True)
        R2   = self.statistic_RS(Tm,method=method, maxfev=maxfev,value = True)

        col1 = np.array(["MAPE","RMSD","AICc","R^2"])
        col2 = np.array([MAPE,RMSD,AIC,R2])
        estadis = pd.DataFrame({"statistics":col1,"values":col2})
        return estadis

    def summary_data(self,Tm,method="lm", maxfev=20000, temp = True, sd =True,name= "Buchowski-Ksiazczak"):
        
        nombre     = name
        values     = self.values(Tm, method = method, maxfev = maxfev, temp = temp,RD =True)
        mapes      = self.statistic_MAPE(Tm,method = method, maxfev = maxfev)
        parameters = self.parameters(Tm,method = method, maxfev = maxfev, sd = sd )
        stadistics = self.statistics(Tm,method = method, maxfev = maxfev)

        DATA = pd.concat([values,mapes,parameters,stadistics], axis=1)
        
        
        url_1 = "/content/sample_data/"+  nombre + "_" +Tm+".xlsx"
        url_3 = "/content/sample_data/"+  nombre + "_" +Tm+".csv"
        url_4 = "/content/sample_data/"+  nombre + "_" +Tm+".png"
        DATA.to_excel(url_1,sheet_name=name)
        DATA.to_csv(url_3)
        return DATA

    def plot(self,Tm,method="lm", maxfev=20000, name = "Buchowski-Ksiazczak"):

        nombre= name

        df_values = self.values(Tm, method=method, maxfev=maxfev, temp = False, RD = False)
        
        url_2 = "/content/sample_data/"+ nombre + "_" +Tm+".pdf"

        fig = go.Figure()
        numerofilas = len(self.temperature_values)
        numerocolumnas = len(self.show.columns)-2
        L = [numerofilas*i for i in range(numerocolumnas+2)]

        X = np.linspace(min(df_values["x_cal"]),max(df_values["x_cal"]),200)

        for i in range(11):
            fig.add_trace(go.Scatter(x=df_values["x_cal"][L[i]:L[i+1]], y=df_values["x_exp"][L[i]:L[i+1]],name=" w"+str(i) ,mode='markers',marker=dict(size=6,line=dict(width=0.5,color='DarkSlateGrey'))))


        fig.add_trace(go.Scatter(x=X,y=X,name="$x_{exp}=x_{cal}$"))


        fig.update_xaxes(title = "$x_{cal}$", title_font=dict(size=30, family='latex', color='rgb(1,21,51)'))
        fig.update_yaxes(title = "$x_{exp}$ ",title_font=dict(size=30,family='latex', color='rgb(1,21,51)'))
        fig.update_layout(title='',title_font=dict(size=26, family='latex', color= "rgb(1,21,51)"),width=1010, height=550)
        fig.update_layout(legend=dict(orientation="h",y=1.2,x=0.03),title_font=dict(size=40, color='rgb(1,21,51)'))
        fig.write_image(url_2)
        fig.write_image(url_4)
        return fig.show()

    def summary_download(self,Tm,name = "Buchowski-Ksiazczak",ext= "xlsx"):
        nombre = name
        extension = ext
        url_1 = "/content/sample_data/"+ nombre + "_" +Tm +"."+extension
        return files.download(url_1)

    def plot_download(self,Tm,name = "Buchowski-Ksiazczak"):
        nombre = name
        url_2 = "/content/sample_data/"+nombre + "_" +Tm +".pdf"
        return files.download(url_2)

def data_upload(name):
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
        
    url = "/content/"+name+".csv"
    return dataframe(url)

In [5]:
datos = "https://raw.githubusercontent.com/josorio398/Datos-Experimentales-Articulos/main/Datos-Experimentales-Progesterona.csv"
tabla=dataframe(datos)
tabla.show

,T,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,278.15,0.003474,0.004135,0.004902,0.005776,0.006768,0.008212,0.009506,0.011551,0.013446,0.016160,0.019426
1,283.15,0.004699,0.005635,0.006558,0.007875,0.009291,0.010839,0.012840,0.015361,0.017629,0.021631,0.025138
2,288.15,0.006434,0.007651,0.008837,0.010590,0.012285,0.014382,0.017283,0.020078,0.023767,0.027730,0.032794
3,293.15,0.008758,0.010016,0.011825,0.014066,0.016600,0.019189,0.022561,0.026279,0.030989,0.035827,0.041767
4,298.15,0.011372,0.013420,0.015643,0.018082,0.021239,0.025000,0.028850,0.033605,0.038841,0.045945,0.054639
5,303.15,0.015176,0.017438,0.020472,0.023678,0.027654,0.032454,0.037721,0.043093,0.050664,0.057556,0.068107
6,308.15,0.019611,0.022677,0.026564,0.030550,0.035239,0.040859,0.047857,0.054909,0.064172,0.074461,0.086796
7,313.15,0.025689,0.029450,0.033628,0.039366,0.044885,0.051327,0.060326,0.070076,0.079765,0.091857,0.107922
8,318.15,0.032771,0.038110,0.043439,0.050537,0.058350,0.067012,0.075074,0.086158,0.101364,0.115677,0.132370


In [6]:
tabla.temperature_values

,T
0,278.15
1,283.15
2,288.15
3,293.15
4,298.15
5,303.15
6,308.15
7,313.15
8,318.15


In [ ]:
tabla.mole_fractions("x0", temp =True )

,T,x0
0,278.15,0.003474
1,283.15,0.004699
2,288.15,0.006434
3,293.15,0.008758
4,298.15,0.011372
5,303.15,0.015176
6,308.15,0.019611
7,313.15,0.025689
8,318.15,0.032771


In [ ]:
tabla.name_mf

['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']

In [ ]:
tabla.experimental_values()

,T,x_exp
0,278.15,0.003474
1,283.15,0.004699
2,288.15,0.006434
3,293.15,0.008758
4,298.15,0.011372
...,...,...
94,298.15,0.054639
95,303.15,0.068107
96,308.15,0.086796
97,313.15,0.107922


In [ ]:
modelo = Buchowski_Ksiazaczak_model(datos)
modelo.equation

<IPython.core.display.Math object>

In [ ]:
modelo.temperature_values

,T
0,278.15
1,283.15
2,288.15
3,293.15
4,298.15
5,303.15
6,308.15
7,313.15
8,318.15


In [ ]:
modelo.mole_fractions("x0", temp =True )

,T,x0
0,278.15,0.003474
1,283.15,0.004699
2,288.15,0.006434
3,293.15,0.008758
4,298.15,0.011372
5,303.15,0.015176
6,308.15,0.019611
7,313.15,0.025689
8,318.15,0.032771


In [ ]:
modelo.experimental_values(temp = False)

,x_exp
0,0.003474
1,0.004699
2,0.006434
3,0.008758
4,0.011372
...,...
94,0.054639
95,0.068107
96,0.086796
97,0.107922


In [ ]:
parametros = modelo.parameters("progesterone","fx", method="lm", maxfev=20000, sd = True)
parametros


,x,λ,σ_λ,h,σ_h
0,x0,0.886642,0.015631,5604.197424,76.654803
1,x1,0.972627,0.018577,5033.697490,74.296979
2,x2,1.084903,0.015930,4466.032359,50.529731
3,x3,1.212516,0.024713,3948.039430,61.797433
4,x4,1.364156,0.037994,3477.892627,74.203938
5,x5,1.503429,0.045155,3109.849775,71.255330
6,x6,1.679510,0.032130,2751.610603,40.023160
7,x7,1.826059,0.032430,2484.425220,33.366440
8,x8,2.197501,0.061281,2077.151788,43.883212
9,x9,2.366302,0.074867,1886.756067,44.929571


In [ ]:
modelo.parameters_h("progesterone")

,x,h
0,x0,5604.197424
1,x1,5033.697490
2,x2,4466.032359
3,x3,3948.039430
4,x4,3477.892627
5,x5,3109.849775
6,x6,2751.610603
7,x7,2484.425220
8,x8,2077.151788
9,x9,1886.756067


In [ ]:
modelo.parameters_λ("progesterone")

,x,λ
0,x0,0.886642
1,x1,0.972627
2,x2,1.084903
3,x3,1.212516
4,x4,1.364156
5,x5,1.503429
6,x6,1.679510
7,x7,1.826059
8,x8,2.197501
9,x9,2.366302


In [ ]:
modelo.calculated_values("progesterone",RD = True)

,T,x_cal,RD
0,278.15,0.003460,0.004127
1,283.15,0.004744,-0.009537
2,288.15,0.006434,-0.000032
3,293.15,0.008637,0.013847
4,298.15,0.011481,-0.009564
...,...,...,...
94,298.15,0.054433,0.003762
95,303.15,0.068948,-0.012344
96,308.15,0.086455,0.003928
97,313.15,0.107326,0.005529


In [ ]:
modelo.values("progesterone", maxfev=20000,temp=False,RD =True)

,x_exp,x_cal,RD
0,0.003474,0.003460,0.004127
1,0.004699,0.004744,-0.009537
2,0.006434,0.006434,-0.000032
3,0.008758,0.008637,0.013847
4,0.011372,0.011481,-0.009564
...,...,...,...
94,0.054639,0.054433,0.003762
95,0.068107,0.068948,-0.012344
96,0.086796,0.086455,0.003928
97,0.107922,0.107326,0.005529


In [ ]:
modelo.statistic_MAPE("progesterone",method="lm", maxfev=20000, temp = True, data= "temperature" )

,T,Temp,MAPE_progesterone
0,T1,278.15,0.812398
1,T2,283.15,0.694098
2,T3,288.15,0.616445
3,T4,293.15,0.925877
4,T5,298.15,0.752180
5,T6,303.15,0.747898
6,T7,308.15,0.570334
7,T8,313.15,0.734715
8,T9,318.15,0.887378


In [ ]:
modelo.statistic_MAPE("progesterone",data = "all")

Mean Absolute Percentage Error, MAPE_progesterone =  0.74903573687629


In [ ]:
modelo.statistic_RMSD("progesterone",method="lm", maxfev=20000)

Root Mean Square Deviation, RMSD_progesterone =  0.0004131060174108147


In [ ]:
modelo.statistic_AIC("progesterone",method="lm", maxfev=20000, corrected=True)

Akaike Information Criterion corrected , AICc_progesterone =  1536.5250152652002


In [ ]:
modelo.statistic_RS("progesterone",method="lm", maxfev=20000)

Coefficient of Determination, R^2_progesterone =  0.99976559672808


In [ ]:
modelo.statistics("progesterone",method="lm", maxfev=20000)

,statistics,values
0,MAPE,0.749036
1,RMSD,0.000413
2,AICc,1536.525015
3,R^2,0.999766


In [ ]:
modelo.summary_data("progesterone",method="lm", maxfev=20000, temp = True, sd =True,name= "Buchowski-Ksiazczak")

,T,x_exp,x_cal,RD,T,Temp,MAPE_progesterone,x,λ,σ_λ,h,σ_h,statistics,values
0,278.15,0.003474,0.003460,0.004127,T1,278.15,0.812398,x0,0.886642,0.015631,5604.197424,76.654803,MAPE,0.749036
1,283.15,0.004699,0.004744,-0.009537,T2,283.15,0.694098,x1,0.972627,0.018577,5033.697490,74.296979,RMSD,0.000413
2,288.15,0.006434,0.006434,-0.000032,T3,288.15,0.616445,x2,1.084903,0.015930,4466.032359,50.529731,AICc,1536.525015
3,293.15,0.008758,0.008637,0.013847,T4,293.15,0.925877,x3,1.212516,0.024713,3948.039430,61.797433,R^2,0.999766
4,298.15,0.011372,0.011481,-0.009564,T5,298.15,0.752180,x4,1.364156,0.037994,3477.892627,74.203938,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,298.15,0.054639,0.054433,0.003762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,303.15,0.068107,0.068948,-0.012344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,308.15,0.086796,0.086455,0.003928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,313.15,0.107922,0.107326,0.005529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
modelo.plot("progesterone",method="lm", maxfev=20000, name = "Buchowski-Ksiazczak")

In [ ]:
modelo.summary_download("progesterone")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
modelo.plot_download("progesterone")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>